# Define a function to evaluate a user result

In [51]:
import pandas as pd
import os
from pathlib import Path
from typing import List, Tuple

def evaluate_candidate_output(ground_truth_path:str, 
                              candidate_name: str, 
                              candidate_output_path: str) -> Tuple[float, List]:
    detailed_results = [] 
    
    # Load answer files
    output_0 = pd.read_csv(os.path.join(candidate_output_path, 'output_0.csv'))
    output_1 = pd.read_csv(os.path.join(candidate_output_path, 'output_1.csv'))
    output_2 = pd.read_csv(os.path.join(candidate_output_path, 'output_2.csv'))

    #### Evaluate the output_1 file ####
    print(f"Evaluating candidate: {candidate_name}")

    # Check if the output_0 has the expected columns # 'Samples', 'No. Males', 'Average Duration', 'SeniorUsers'
    is_output_0_correct = output_0.columns.tolist() == ['Samples', 'No. Males', 'Average Duration', 'SeniorUsers']
    score_output_0 = 0
    if is_output_0_correct:
        print("output_0 - columns are correct")
        print("restul1", output_0["Samples"].iloc[0])
        if output_0["Samples"].iloc[0] == 9000:
            detailed_results.append(10)
            score_output_0 += detailed_results[-1]
            print("Samples column is correct")

        if output_0["No. Males"].iloc[0] == 4443:
            detailed_results.append(10)
            score_output_0 += detailed_results[-1]
            print("No. Males column is correct")

        if output_0["Average Duration"].iloc[0] == 15.51:
            detailed_results.append(10)
            score_output_0 += detailed_results[-1]
            print("Average Duration column is correct")

        if output_0["SeniorUsers"].iloc[0] == 412:
            detailed_results.append(10)
            score_output_0 += detailed_results[-1]
            print("SeniorUsers column is correct")
    else:
            detailed_results.extend([0]*4)

    ### Evaluate the output_1 file ####
    score_output_1 = 0
    is_output_1_correct = output_1.columns.tolist() == ["Calories"]
    if is_output_1_correct:
        print("output_1 columns are correct")

        # Load the dataset_eval_t dataset that contains the column Calories then compare it with the output_2 price column using MAE metric
        # Do the mae for each row then sum all the mae and divide it by the number of rows
        dataset_eval_t = pd.read_csv(os.path.join(ground_truth_path, 'task1_dataset_eval_t.csv'))

        mae = (dataset_eval_t["Calories"] - output_1["Calories"]).abs().sum() / len(dataset_eval_t)
        print("MAE: ", mae)

        # Assign scores based on MAE value
        import bisect
        ranges = [8.5, 9, 10, 12]
        scores = [40, 30, 20, 10]

        index = bisect.bisect_left(ranges, mae)
        if index < len(scores):
            score_output_1 += scores[index]

        detailed_results.append(score_output_1)

        print("Took the score: ", score_output_1)
    else:
        detailed_results.append(0)

    score_output_2 = 0
    is_output_2_correct = output_2.columns.tolist() == ["Calories"]
    if is_output_2_correct:
        print("output_2 columns are correct")

        # Load the dataset_eval_t dataset that contains the column Calories then compare it with the output_2 price column using MAE metric
        # Do the mae for each row then sum all the mae and divide it by the number of rows
        dataset_eval_t = pd.read_csv(os.path.join(ground_truth_path, 'task2_dataset_eval_t.csv'))

        mae = (dataset_eval_t["Calories"] - output_2["Calories"]).abs().sum() / len(dataset_eval_t)
        print("MAE: ", mae)

        # Assign scores based on MAE value
        import bisect
        ranges = [18, 23, 26, 30]
        scores = [20, 15, 10, 5]

        index = bisect.bisect_left(ranges, mae)
        if index < len(scores):
            score_output_2 += scores[index]

        detailed_results.append(score_output_2)

        print("Took the score: ", score_output_2)
    else:
        detailed_results.append(0)


    # Calculate the total score
    total_score = score_output_0 + score_output_1 + score_output_2
    print("Total score: ", total_score)

    return total_score, detailed_results

# Then go through all and create a csv files with results

In [52]:
# Take all candidates and evaluate them. Create a final csv with outputs 
from pathlib import Path


# Open the outputs from candidates
candidates_path = Path("../Solutii")
folder_candidati = [f.name for f in candidates_path.iterdir() if f.is_dir()]
print(folder_candidati)

# Open each candidates and run the evaluator
candidate_results = []
for candidate_name in folder_candidati:
    if candidate_name[0] == '.':
        continue 
        
    candidat_path = candidates_path / candidate_name
    
    relative_path = str(Path(candidat_path))
    total_score, detailed_results = evaluate_candidate_output(ground_truth_path="Dataset",
                              candidate_name=candidate_name,
                              candidate_output_path=relative_path)

    candidate_results.append({'Name' : candidate_name, 'Detailed': detailed_results, 'Total' : total_score})

df = pd.DataFrame(candidate_results)
df = df.sort_values(by="Total", ascending=False)  # Sort in descending order (highest score first)
df.to_csv(candidates_path/ "rezultate.csv", index=False)
    
df


['Candidat_avansat', 'Candidat_incepator']
Evaluating candidate: Candidat_avansat
output_0 - columns are correct
restul1 9000
Samples column is correct
No. Males column is correct
Average Duration column is correct
SeniorUsers column is correct
output_1 columns are correct
MAE:  8.360868935370371
Took the score:  40
output_2 columns are correct
MAE:  17.735515802924297
Took the score:  20
Total score:  100
Evaluating candidate: Candidat_incepator
output_0 - columns are correct
restul1 9000
Samples column is correct
No. Males column is correct
Average Duration column is correct
SeniorUsers column is correct
output_1 columns are correct
MAE:  9.30688678275
Took the score:  20
output_2 columns are correct
MAE:  27.102977750999997
Took the score:  5
Total score:  65


,Name,Detailed,Total
0,Candidat_avansat,"[10, 10, 10, 10, 40, 20]",100
1,Candidat_incepator,"[10, 10, 10, 10, 20, 5]",65
